In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

model_dir = "fine_tuned_lora_llama"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the base model
model = AutoModelForCausalLM.from_pretrained(model_dir)

model = PeftModel.from_pretrained(model, model_dir)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Fine-tuned model and tokenizer loaded successfully!")


Fine-tuned model and tokenizer loaded successfully!


In [5]:
def generate_text(model, tokenizer, prompt, max_length=150):
    """
    Generate text using the fine-tuned model.
    """
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(
        inputs["input_ids"], 
        attention_mask=inputs["attention_mask"], 
        max_length=max_length, 
        num_return_sequences=1,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "Write a detailed job description for a Data Scientist role. He has to"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text:
Write a detailed job description for a Data Scientist role. He has to be an expert in data science and machine learning. He has to be able to understand and interpret data and make intelligent decisions based on the data. He has to be able to interpret the data and make intelligent decisions based on the data. He has to be able to interpret the data and make intelligent decisions based on the data. He has to be able to interpret the data and make intelligent decisions based on the data. He has to be able to interpret the data and make intelligent decisions based on the data. He has to be able to interpret the data and make intelligent decisions based on the data. He has to be able to interpret the data and make intelligent decisions based on the
